<a href="https://colab.research.google.com/github/mvmarcio/Analise_Dados_Spark/blob/main/Treinamento%20PySpark%20Parte%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Análise de Dados com PySpark**

O objeteivo deste documento é realizar uma análise de dados utilizando a linguagem PySpark. Serão realizadas as seguintes análises:



*   Instalação PySpark
*   Importação bibliotecas / funções
*   Criar Sessão PySpark
*   Criar data frame / ler arquivo
*   Exibir data frame
*   Verificar tipos de colunas
*   Verificando dados nulos
*   Selecionar Colunas
*   Selecionar Colunas com ALIAS
*   Organizar Select
*   Filtrar o data frame
*   Filtrar o data frame com 2 ou mais condições (AND / &)
*   Filtrar o data frame com 2 ou mais condições (OR / |)
*   Filtrar o data frame combinando & e | (And e OR)
*   Criar novas colunas (usando função lit)
*   Criar coluna condicional (usando função substring)
*   Criar coluna condicional (usando função concat_ws)
*   Renomear Colunas
*   Alterar tipo de coluna
*   Window Function 1 - Numero de linhas - row_number()
*   Window Function 2 - Ranking 1 - rank()
*   Window Function 3 - Ranking 2 - dense_rank()
*   Window Function 4 - Porcentagem Ranking - percent_rank()
*   Window Function 5 - Divisão em ' N ' partes - ntile()
*   Window Function 6 - LAG / Degrau - lag()
*   Window Function 7 - Lead / Degrau - lead()
*   Agregações
*   GroupBy + AGG 1
*   Where
*   Describe
*   Window Function 8 - Função de agregação usando Window Function



A análise será realizada com os dados dos jogadores da Copa do Mundo de 2018 e está disponível neste repositório ou neste link [Copa do Mundo 2018](https://https://www.kaggle.com/datasets/djamshed/fifa-world-cup-2018-players).



O PySpark é a interface Python para o Apache Spark, que é um motor de processamento de dados em larga escala que permite realizar computação em clusters de forma distribuída. O PySpark facilita o uso do Spark com a linguagem Python, o que permite escrever os códigos e análises em Python enquanto aproveitam a potência e a escalabilidade do Spark.

### **1. Instalando a Biblioteca do PySpark**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=5e7f00010250bc3200ac331658db8be8c65fea42d5f8b05f414ba972b8381bcb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# Vamos chamar a biblioteca externa
from pyspark.sql import SparkSession # para gerar o cluster

from pyspark.sql.functions import * # importa as funções SQL Spark

from pyspark.sql.types import * # Suporte para data frames

from pyspark.sql.window import Window # Importando window function

### **2. Vamos Criar o Cluster para a Análise**

Para detalhes dessa funcionalidade acessar o link [PySpark](https://https://spark.apache.org/docs/latest/sql-getting-started.html)

In [3]:
# Criando nosso cluster e sessão
spark = (
    SparkSession.builder
    .master('local') # cluster local
    .appName('PySpark_01') # nome de usuario
    .getOrCreate()
)

### **3. Vamos Ler Nosso Arquivo da Análise**

In [4]:
# Lendo arquivo
dados = spark.read.csv('wc2018-players.csv', header=True, inferSchema=True)
# O argumento header é para dizer que nossos dados possuem cabeçalho
# O argumento inferSchema é para que os dados sejam automaticamente editados como numéricos ou strings. Se não for colocado todos são lidos como strings

dados.show(5) # mostrando as primeiras 5 linhas

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [5]:
# Vamos verificar os tipos das variáveis
dados.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



In [6]:
# Vamos renomear as colunas com a função withColumnRenamed
dados = dados.withColumnRenamed('Pos.', 'Posicao') \
.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Shirt Name', 'Nome Camiseta').withColumnRenamed('Club', 'Time').withColumnRenamed('Height', 'Altura')\
.withColumnRenamed('Weight', 'Peso')

dados.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [7]:
# Vamos verificar os valores nulos
for colunas in dados.columns:
  print(colunas, dados.filter(dados[colunas].isNull()).count())

Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


In [8]:
# Vamos selecionar colunas do data frame com a função select
dados.select('Selecao', 'Altura').show(5)

+---------+------+
|  Selecao|Altura|
+---------+------+
|Argentina|   169|
|Argentina|   169|
|Argentina|   167|
|Argentina|   167|
|Argentina|   170|
+---------+------+
only showing top 5 rows



In [9]:
 # Vamos filtrar dados da seleção brasileira
 dados.filter(dados.Selecao == 'Brazil').show(5) # exibe as 5 primeiras linhas do Brasil

+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|  Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|       FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|     TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|     FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|     NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [10]:
# Vamos filtrar a seleção brasileira com outra função
dados.filter('Selecao == "Brazil" ').show(5) # exibe as primeiras 5 linhas

+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|  Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|       FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|     TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|     FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|     NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
+-------+------+-------+-----------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [11]:
# Vamos filtrar duas condições: jogadores da seleção da Argentina acima de 1,80 m
dados.filter((dados.Selecao == 'Argentina') & (dados.Altura > 180)).show()

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|
|Argentina|     9|     FW|   HIGUAIN Gonzalo|10.12.1987|      HIGUAÍN|   Juventus FC (ITA)|   184|  75|
|Argentina|    23|     GK|CABALLERO Wilfredo|28.09.1981|    CABALLERO|    Chelsea FC (ENG)|   186|  80|
|Argentina|     2|     DF|   MERCADO Gabriel|18.03.1987|      MERCADO|    Sevilla FC (ESP)|   181|  81|
|Argentina|    17|     DF|  OTAMENDI Nicolas|12.02.1988|     OTAMENDI|Manchester City F...|   181|  81|
|Argentina|    16|     DF|       ROJO Marcos|20.03.1990|         ROJO|Manchester United...|   189|  82|
|Argentina|     6|     DF|    FAZIO Federico|17.03.1987|        

In [12]:
# Vamos filtrar duas condições: jogadores da seleção do Brasil acima de 85 kg
dados.filter((dados.Selecao == 'Brazil') & (dados.Peso > 85)).show()

+-------+------+-------+--------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|     Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+--------------+----------+-------------+--------------------+------+----+
| Brazil|     8|     MF|RENATO AUGUSTO|08.02.1988|   R. AUGUSTO| Beijing Guoan (CHN)|   186|  86|
| Brazil|    23|     GK|       EDERSON|17.08.1993|      EDERSON|Manchester City F...|   188|  86|
| Brazil|     1|     GK|       ALISSON|02.10.1992|    A. BECKER|       AS Roma (ITA)|   193|  91|
| Brazil|    16|     GK|        CASSIO|06.06.1987|       CASSIO|SC Corinthians (BRA)|   195|  92|
+-------+------+-------+--------------+----------+-------------+--------------------+------+----+



In [13]:
# Vamos filtrar os jogadores acima de 1,85 do Brasil e México
dados.filter((dados.Altura > 185) & ((dados.Selecao == 'Brazil') | (dados.Selecao == 'Mexico'))).show()

+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
| Brazil|     3|     DF|         MIRANDA|07.09.1984|      MIRANDA|FC Internazionale...|   186|  78|
| Brazil|     4|     DF|   PEDRO GEROMEL|21.09.1985|      GEROMEL|   Grêmio FBPA (BRA)|   190|  84|
| Brazil|     8|     MF|  RENATO AUGUSTO|08.02.1988|   R. AUGUSTO| Beijing Guoan (CHN)|   186|  86|
| Brazil|    23|     GK|         EDERSON|17.08.1993|      EDERSON|Manchester City F...|   188|  86|
| Brazil|     1|     GK|         ALISSON|02.10.1992|    A. BECKER|       AS Roma (ITA)|   193|  91|
| Brazil|    16|     GK|          CASSIO|06.06.1987|       CASSIO|SC Corinthians (BRA)|   195|  92|
| Mexico|    21|     FW|   ALVAREZ Edson|24.10.1997|   E. ÁLVAREZ|  Club América (MEX)|   186|  72|


In [14]:
# Vamos criar uma nova coluna usando a função withColumn. Coluna do ano 2018
dados.withColumn('Ano', lit(2018)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|2018|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|2018|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|2018|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|2018|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|2018|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
only showing top 5 

In [15]:
# Vamos criar uma coluna nova com a abrevição do nome da seleção
dados.withColumn('Sigla', substring(dados.Selecao, 1, 3)).show(5) # O Spark comça do 1 igual o R

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Sigla|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|  Arg|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|  Arg|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|  Arg|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|  Arg|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|  Arg|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-----+
only showi

In [16]:
# Vamos pegar o ano de nascimento dos jogadores
# A função pega os últimos quatro dígitos (-4) e vai andar quatro casas (4)
dados.withColumn('Ano_Nascimento', substring(dados.Nascimento, -4, 4)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|          1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|          1987|
+---------+------+-------+------------------+---

In [17]:
# Vamos salvar a consulta anterior
dados = dados.withColumn('Ano_Nascimento', substring(dados.Nascimento, -4, 4))
dados.filter(dados.Selecao == 'Denmark').show(5)

+-------+------+-------+-------------------+----------+-------------+--------------------+------+----+--------------+
|Selecao|Numero|Posicao|          Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|
+-------+------+-------+-------------------+----------+-------------+--------------------+------+----+--------------+
|Denmark|     2|     MF|KROHN-DEHLI Michael|06.06.1983|  KROHN-DEHLI|Deportivo La Coru...|   171|  69|          1983|
|Denmark|    23|     FW|        SISTO Pione|04.02.1995|        SISTO|    Celta Vigo (ESP)|   173|  69|          1995|
|Denmark|    11|     FW| BRAITHWAITE Martin|05.06.1991|  BRAITHWAITE|FC Girondins Bord...|   180|  73|          1991|
|Denmark|    10|     MF|  ERIKSEN Christian|14.02.1992|      ERIKSEN|Tottenham Hotspur...|   181|  76|          1992|
|Denmark|     8|     MF|     DELANEY Thomas|03.09.1991|      DELANEY|SV Werder Bremen ...|   180|  78|          1991|
+-------+------+-------+-------------------+----------+-

In [18]:
# Vamos concatenar o nome da camisa e o número
dados.withColumn('Nome_camisa', concat_ws('-', 'Numero', 'Nome Camiseta')).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento| Nome_camisa|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992|3-TAGLIAFICO|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996|    22-PAVÓN|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993|  15-LANZINI|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|          1990|   18-SALVIO|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC B

In [19]:
# Vamos transformar a coluna ano para data. Ela está como string
print(dados.printSchema()) # visualizando o tipo das colunas
print()
dados.withColumn('Ano_Nascimento', col('Ano_Nascimento').cast(IntegerType())).printSchema() # transformando a coluna Ano_Nascimento

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano_Nascimento: string (nullable = true)

None

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano_Nascimento: integer (nullable = true)



In [20]:
# Vamos salvar os dados com a coluna corrigida
dados = dados.withColumn('Ano_Nascimento', col('Ano_Nascimento').cast(IntegerType()))
dados.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano_Nascimento: integer (nullable = true)



In [21]:
# Corrigindo a data que está em string para data

## Vamos fazer uma função "cortar" do dia
dia = udf(lambda data: data.split('.')[0])

dados.withColumn('Dia', dia(dados.Nascimento)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992| 31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996| 21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993| 15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|          1990| 13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|          1987| 24|
+---------+-----

In [22]:
# Vamos salvar no data frame
dados = dados.withColumn('Dia', dia(dados.Nascimento))
dados.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992| 31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996| 21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993| 15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|          1990| 13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|          1987| 24|
+---------+-----

In [23]:
# Vamos fazer uma função para cortar o mês
mes = udf(lambda data: data.split('.')[1])
dados = dados.withColumn('Mes', mes(dados.Nascimento))
dados.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992| 31| 08|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996| 21| 01|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993| 15| 02|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|          1990| 13| 07|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|        

In [24]:
# Vamos criar a variável Data_Nascimento
dados = dados.withColumn('Data_Nascimento', concat_ws('-', 'Ano_Nascimento', 'Mes', 'Dia'))
dados.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992| 31| 08|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996| 21| 01|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993| 15| 02|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|          1990| 13| 07|     1990-07-13|

In [25]:
# Vamos transormar a coluna Data_Nascimento em tipo data
dados.withColumn('Data_Nascimento', to_date(dados.Data_Nascimento, 'yyyy-MM-dd')).printSchema()

# Salvando os dados
dados = dados.withColumn('Data_Nascimento', to_date(dados.Data_Nascimento, 'yyyy-MM-dd'))

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano_Nascimento: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Data_Nascimento: date (nullable = true)



In [26]:
# Vamos calcular a idade dos jogadores
dados.withColumn('Idade', (year(current_date()) - dados.Ano_Nascimento)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992| 31| 08|     1992-08-31|   32|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996| 21| 01|     1996-01-21|   28|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993| 15| 02|     1993-02-15|   31|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|   

In [27]:
# Vamos salvar as idades
dados = dados.withColumn('Idade', (year(current_date()) - dados.Ano_Nascimento))
dados.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992| 31| 08|     1992-08-31|   32|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996| 21| 01|     1996-01-21|   28|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|          1993| 15| 02|     1993-02-15|   31|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|   

In [28]:
# Vamos apagar a coluna nascimento

dados = dados.drop('Nascimento')
dados.show(5)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|          1992| 31| 08|     1992-08-31|   32|
|Argentina|    22|     MF|    PAVON Cristian|        PAVÓN|CA Boca Juniors (...|   169|  65|          1996| 21| 01|     1996-01-21|   28|
|Argentina|    15|     MF|    LANZINI Manuel|      LANZINI|West Ham United F...|   167|  66|          1993| 15| 02|     1993-02-15|   31|
|Argentina|    18|     DF|    SALVIO Eduardo|       SALVIO|    SL Benfica (POR)|   167|  69|          1990| 13| 07|     1990-07-13|   34|
|Argentina|    10|     FW|      ME

In [29]:
# Vamos criar um data frame reserva
dados2 = dados

In [30]:
# Vamos fazer um agrupamento por seleção do maior para o menor
num_linha = Window.partitionBy('Selecao').orderBy(col('Altura').desc())

dados.withColumn('Row_Number', row_number().over(num_linha)).show(20)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+----------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|Row_Number|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+----------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|          1987| 17| 03|     1987-03-17|   37|         1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|         2|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82|          1990| 20| 03|     1990-03-20|   34|         3|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85|          1986

In [31]:
# Vamos fazer os rankings de altura dos jogadores por seleção

rank1 = Window.partitionBy('Selecao').orderBy(col('Altura').desc())

dados.withColumn('Rank', rank().over(rank1)).show(20)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|Rank|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+----+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|          1987| 17| 03|     1987-03-17|   37|   1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|   2|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82|          1990| 20| 03|     1990-03-20|   34|   3|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85|          1986| 16| 10|     1986-10-16|   38|   3|

In [32]:
# Vamos fazer um novo ranking. Agora a função não faz contagem de linha
rank2 = Window.partitionBy('Selecao').orderBy(col('Altura').desc())

dados.withColumn('Rank2', dense_rank().over(rank2)).show(20)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+-----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|Rank2|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+-----+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|          1987| 17| 03|     1987-03-17|   37|    1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|    2|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82|          1990| 20| 03|     1990-03-20|   34|    3|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85|          1986| 16| 10|     1986-10-16|   38

In [33]:
# Vamos realizar o ranking por porcentagem
porcentagem = Window.partitionBy('Selecao').orderBy(col('Altura').desc())

dados.withColumn('Porcentagem', percent_rank().over(porcentagem)).show(20)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+--------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|         Porcentagem|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+--------------------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|          1987| 17| 03|     1987-03-17|   37|                 0.0|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|0.045454545454545456|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82|          1990| 20| 03|     1990-03-20|   34| 0.09090909090909091|
|Argentina|    12|     GK|     ARMANI Franco| 

In [34]:
# Vamos criar uma coluna que divide os jogadores em grupos pela altura
# Vamos dividir em 5 grupos

parte = Window.partitionBy('Selecao').orderBy(col('Altura').desc())

dados.withColumn('Grupo', ntile(5).over(parte)).show(20)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+-----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|Grupo|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+-----+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|          1987| 17| 03|     1987-03-17|   37|    1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|    1|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82|          1990| 20| 03|     1990-03-20|   34|    1|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85|          1986| 16| 10|     1986-10-16|   38

In [36]:
# Função lag()

degrau = Window.partitionBy('Selecao').orderBy(col('Altura').desc())

dados.withColumn('degrau', lag('Altura', 1).over(degrau)).show(20)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|degrau|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|          1987| 17| 03|     1987-03-17|   37|  NULL|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|   199|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82|          1990| 20| 03|     1990-03-20|   34|   192|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85|          1986| 16| 10|     1986-10-16

In [37]:
# Função degrau com o lead pegando da posterior
degrau2 = Window.partitionBy('Selecao').orderBy(col('Altura').desc())

dados.withColumn('degrau2', lead('Altura', 1).over(degrau2)).show(30)

+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+-------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|degrau2|
+---------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+-------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|          1987| 17| 03|     1987-03-17|   37|    192|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|    189|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82|          1990| 20| 03|     1990-03-20|   34|    189|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85|          1986| 16| 10|     1986

In [40]:
# Comando group by

dados.groupBy('Selecao').agg({'Altura': 'avg'}).orderBy('avg(Altura)', ascending = False).show(32)

+--------------+------------------+
|       Selecao|       avg(Altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|    Costa Rica|180.69565217

In [41]:
# Outra forma de fazer o group by
dados.groupBy('Selecao').agg(avg('Altura')).orderBy('avg(Altura)', ascending = False).show(32)

+--------------+------------------+
|       Selecao|       avg(Altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|    Costa Rica|180.69565217

In [42]:
# Vamos verificar o jogador mais alto
dados.groupBy('Selecao').agg(max('Altura')).orderBy('max(Altura)', ascending = False).show(32)

+--------------+-----------+
|       Selecao|max(Altura)|
+--------------+-----------+
|       Croatia|        201|
|       Denmark|        200|
|     Argentina|        199|
|       Belgium|        199|
|        Sweden|        198|
|       Iceland|        198|
|        France|        197|
|       Nigeria|        197|
|Korea Republic|        197|
|        Panama|        197|
|        Russia|        196|
|       Senegal|        196|
|       Uruguay|        196|
|       England|        196|
|    Costa Rica|        196|
|       Germany|        195|
|        Brazil|        195|
|        Poland|        195|
|        Serbia|        195|
|       IR Iran|        194|
|         Spain|        194|
|         Egypt|        194|
|      Colombia|        194|
|     Australia|        193|
|       Tunisia|        192|
|  Saudi Arabia|        192|
|   Switzerland|        192|
|      Portugal|        191|
|       Morocco|        190|
|        Mexico|        190|
|          Peru|        189|
|         Japa

In [45]:
# Vamos trabalhar com a função where
dados.where(dados.Selecao == 'Brazil').where(dados.Posicao == 'DF').show()

+-------+------+-------+-------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
|Selecao|Numero|Posicao|    Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|
+-------+------+-------+-------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+
| Brazil|    22|     DF|       FAGNER|       FAGNER|SC Corinthians (BRA)|   168|  67|          1989| 11| 06|     1989-06-11|   35|
| Brazil|     6|     DF|  FILIPE LUIS|  FILIPE LUIS|Atletico Madrid (...|   182|  73|          1985| 09| 08|     1985-08-09|   39|
| Brazil|    13|     DF|   MARQUINHOS|   MARQUINHOS|Paris Saint-Germa...|   183|  75|          1994| 14| 05|     1994-05-14|   30|
| Brazil|     3|     DF|      MIRANDA|      MIRANDA|FC Internazionale...|   186|  78|          1984| 07| 09|     1984-09-07|   40|
| Brazil|    14|     DF|       DANILO|       DANILO|Manchester City F...|   184|  7

In [47]:
# Vamos verificar os jogadores mais pesados
top1 = Window.partitionBy('Selecao').orderBy(col('Peso').desc())

dados.withColumn('topo', rank().over(top1)).where('topo = "1" ').show()

+--------------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+----+
|       Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Ano_Nascimento|Dia|Mes|Data_Nascimento|Idade|topo|
+--------------+------+-------+------------------+-------------+--------------------+------+----+--------------+---+---+---------------+-----+----+
|     Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|          1986| 10| 02|     1986-02-10|   38|   1|
|     Australia|    18|     GK|     VUKOVIC Danny|      VUKOVIC|      KRC Genk (BEL)|   187|  94|          1985| 27| 03|     1985-03-27|   39|   1|
|       Belgium|     9|     FW|     LUKAKU Romelu|    R. LUKAKU|Manchester United...|   190|  94|          1993| 13| 05|     1993-05-13|   31|   1|
|        Brazil|    16|     GK|            CASSIO|       CASSIO|SC Corinthians (BRA)|   195|  92|          1987|

In [48]:
# Vamos fazer as estatísticas descritivas do banco de dados
dados.describe().show()

+-------+---------+-----------------+-------+------------+-------------+--------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+
|summary|  Selecao|           Numero|Posicao|   Nome_FIFA|Nome Camiseta|                Time|           Altura|             Peso|    Ano_Nascimento|               Dia|               Mes|             Idade|
+-------+---------+-----------------+-------+------------+-------------+--------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+
|  count|      736|              736|    736|         736|          736|                 736|              736|              736|               736|               736|               736|               736|
|   mean|     NULL|             12.0|   NULL|        NULL|         NULL|                NULL|182.4076086956522|77.18885869565217| 1990.110054347826|15.793478260869565|5.8790760

In [49]:
# Vamos fazer as estatísticas descritivas do Brazil
dados.where(dados.Selecao == 'Brazil').describe().show()

+-------+-------+-----------------+-------+---------+-------------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+
|summary|Selecao|           Numero|Posicao|Nome_FIFA|Nome Camiseta|                Time|           Altura|             Peso|   Ano_Nascimento|              Dia|              Mes|             Idade|
+-------+-------+-----------------+-------+---------+-------------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+
|  count|     23|               23|     23|       23|           23|                  23|               23|               23|               23|               23|               23|                23|
|   mean|   NULL|             12.0|   NULL|     NULL|         NULL|                NULL|180.7826086956522|76.56521739130434|1989.391304347826|11.26086956521739|6.130434782608695|34.608695652173914|
| stddev| 

In [51]:
# Vamos fazer as médias das alturas por times
parametro = Window.partitionBy('Selecao').orderBy(col('Altura').desc())
parametro2 = Window.partitionBy('Selecao')

dados.withColumn('linha', row_number().over(parametro))\
.withColumn('media', avg('Altura').over(parametro2))\
.withColumn('max', max('Altura').over(parametro2))\
.withColumn('mediana', median('Altura').over(parametro2))\
.withColumn('min', min('Altura').over(parametro2))\
.withColumn('std', stddev('Altura').over(parametro2))\
.where('linha = "1" ').select('Selecao', 'media', 'mediana', 'min', 'max', 'std')\
.orderBy('media', ascending = False).show(20)

+--------------+------------------+-------+---+---+------------------+
|       Selecao|             media|mediana|min|max|               std|
+--------------+------------------+-------+---+---+------------------+
|        Serbia|186.69565217391303|  187.0|169|195| 6.189689246192162|
|       Denmark| 186.6086956521739|  187.0|171|200| 7.371561635679147|
|       Germany| 185.7826086956522|  186.0|176|195| 5.247999807191742|
|        Sweden| 185.7391304347826|  185.0|177|198|5.5531922948905175|
|       Iceland|185.52173913043478|  186.0|170|198|   5.6235670111464|
|       Belgium|185.34782608695653|  186.0|169|199| 7.595505223798012|
|       Croatia| 185.2608695652174|  186.0|172|201| 6.173384155935763|
|       Nigeria|184.52173913043478|  185.0|172|197| 6.940458232300529|
|       IR Iran|184.47826086956522|  185.0|177|194| 4.925532414391097|
|        Russia| 184.3913043478261|  185.0|173|196| 5.458430763796681|
|       Senegal|183.65217391304347|  184.0|173|196|7.3274620166018485|
|     